Используя данные на страницах https://стопкоронавирус.рф/information/,  http://www.statdata.ru/nasel_regions  отобразить на графике №1 процент заболевших за последний день людей в зависимости от плотности населения для каждого региона. На графике №2 отобразить "среднюю" вероятность заболеть (% в день) для каждого региона на карте (России) и найти (распечатать) самый безопасный по этому критерию регион.
График №1: по горизонтальной оси -  плотность населения (чел/км2); по вертикальной оси - % заболевших за день. 
Для графика №1 использовать для подсчёта % заболевших колонку «Новые» (см. рисунок ниже)
График №2 (карта России): при наведении на регион должна отображаться вероятность заболеть в %
"Средняя" вероятность заболеть = заболевшие за день/количество населения * 100%


In [1]:
#Импортируем нужные библиотеки
import requests
import json
import pandas as pd
import plotly.express as px

In [2]:
#Парсим сайт стопкоронавирус

html = requests.get('https://xn--80aesfpebagmfblc0a.xn--p1ai/information/').text

start = '<cv-spread-overview :spread-data='  # Начало обрезки
end = "isolation-data"  # Конец обрезки

raw_json_data = html[html.find(start) + 34:html.find(end) - 3]  # Вырезаем json из html страницы

json_data = json.loads(raw_json_data)

city = []

for item in json_data:

    title = item['title']  # Город
    sick = item['sick']  # Выявлено
    healed = item['healed']  # Выздоровело
    died = item['died']  # Умерло
    sick_incr = item['sick_incr']  # Новые
    healed_incr = item['healed_incr']  # В ыздоровело за сутки
    died_incr = item['died_incr']  # Умерло за сутки

#Добавляем в список полученные значения   
    city.append(title)
    city.append(sick)
    city.append(healed)
    city.append(died)
    city.append(sick_incr)
    city.append(healed_incr)
    city.append(died_incr)

#Убираем лишнее
def split(arr, size):
    arrs = []
    while len(arr) > size:
        pice = arr[:size]
        arrs.append(pice)
        arr = arr[size:]
    arrs.append(arr)
    return arrs

a = (split(city, 7))

In [3]:
#Создаём DataFrame из данных с сайта стопкоронавирус
tr = pd.DataFrame(a)
tr = tr.rename(columns={ tr.columns[0]: "city", tr.columns[1]: "identified", 
                         tr.columns[2]: "recovered", tr.columns[3]: "died", 
                         tr.columns[4]: "new", tr.columns[5]: "recovered_day", 
                         tr.columns[6]: "died_day" })
pd.options.display.max_rows = 15
tr

,city,identified,recovered,died,new,recovered_day,died_day
0,Москва,1530989,1395285,26309,1639,3072,61
1,Санкт-Петербург,547704,519868,19757,1811,1972,49
2,Московская область,400007,330540,7491,1309,1922,29
3,Нижегородская область,145993,132727,5325,537,425,25
4,Ростовская область,114218,100373,5412,479,374,26
...,...,...,...,...,...,...,...
80,Республика Марий Эл,16688,15047,413,61,51,1
81,Магаданская область,9775,8895,179,29,18,2
82,Еврейская автономная область,6125,4636,188,39,37,0
83,Ненецкий автономный округ,1694,1592,17,10,2,0


In [4]:
#Создаём DataFrame из данных с сайта www.statdata.ru 
dr = pd.read_html('http://www.statdata.ru/nasel_regions', header=0)[6]
dr = dr.rename(columns={ dr.columns[1]: "city", dr.columns[2]: "population_density", 
                         dr.columns[3]: "population", dr.columns[4]: "area", 
                         dr.columns[5]: "federal district" })
dr = dr.iloc[:, 1:6]
dr

,city,population_density,population,area,federal district
0,Москва,"4 956,06",12 692 466,2 561,ЦФО
1,Санкт-Петербург,"3 843,90",5 392 992,1 403,СЗФО
2,Севастополь,51948,448 829,864.0,ЮФО
3,Московская область,17342,7 687 647,44 329,ЦФО
4,Республика Ингушетия,16224,506 688,3 123,СКФО
...,...,...,...,...,...
83,Камчатский край,067,312 438,464 275,ДВФО
84,Республика Саха (Якутия),031,970 105,3 083 523,ДВФО
85,Магаданская область,030,140 199,462 464,ДВФО
86,Ненецкий автономный округ,025,44 110,176 810,СЗФО


In [5]:
#Объединяем DataFrame
new_tr = tr.merge(dr, how="inner", left_index=False, right_index=False)

In [6]:
# total - рассчитываем, "Средняя" вероятность заболеть = заболевшие за день/количество населения * 100%
new_tr = new_tr.replace(r'\s+','',regex=True)
new_tr['population'] = pd.to_numeric(new_tr['population'].str.replace('\D', ''), errors='coerce')
new_tr['total'] = (new_tr.new / new_tr.population)*100
new_tr

,city,identified,recovered,died,new,recovered_day,died_day,population_density,population,area,federal district,total
0,Москва,1530989,1395285,26309,1639,3072,61,"4956,06",12692466,2561,ЦФО,0.012913
1,Санкт-Петербург,547704,519868,19757,1811,1972,49,"3843,90",5392992,1403,СЗФО,0.033581
2,Московскаяобласть,400007,330540,7491,1309,1922,29,17342,7687647,44329,ЦФО,0.017027
3,Нижегородскаяобласть,145993,132727,5325,537,425,25,4181,3203818,76624,ПФО,0.016761
4,Ростовскаяобласть,114218,100373,5412,479,374,26,4155,4195327,100967,ЮФО,0.011417
...,...,...,...,...,...,...,...,...,...,...,...,...
68,РеспубликаАдыгея,16870,15432,305,61,15,2,5948,463453,7792,ЮФО,0.013162
69,РеспубликаМарийЭл,16688,15047,413,61,51,1,2905,679094,23375,ПФО,0.008983
70,Магаданскаяобласть,9775,8895,179,29,18,2,030,140199,462464,ДВФО,0.020685
71,Ненецкийавтономныйокруг,1694,1592,17,10,2,0,025,44110,176810,СЗФО,0.022671


In [33]:
#percentage got sick
pgs = new_tr[['city', 'population_density', 'new']]
pgs = pgs.replace(r'\s+','',regex=True)
pgs['population_density'] = pd.to_numeric(pgs['population_density'].str.replace('\D', ''), errors='coerce')
pgs['pgsp'] = (pgs.new / pgs.population_density)*100
pgs


,city,population_density,new,pgsp
0,Москва,495606,1639,0.330706
1,Санкт-Петербург,384390,1811,0.471136
2,Московскаяобласть,17342,1309,7.548149
3,Нижегородскаяобласть,4181,537,12.843817
4,Ростовскаяобласть,4155,479,11.528279
...,...,...,...,...
68,РеспубликаАдыгея,5948,61,1.025555
69,РеспубликаМарийЭл,2905,61,2.099828
70,Магаданскаяобласть,30,29,96.666667
71,Ненецкийавтономныйокруг,25,10,40.000000


In [34]:
fig = px.histogram(pgs, x="city", y="pgsp")
fig.show()

In [36]:
#Строим на графике процент заболевших за последний день людей в зависимости от плотности населения для каждого региона.
#по горизонтальной оси -  плотность населения (чел/км2); по вертикальной оси - количество заболевших за день. 

fig = px.histogram(new_tr, x="population_density", y="new", hover_name="city")
fig.show()




In [8]:
#Добавляем геоанные регионов
city_loc = [['Москва', 55.7540471, 37.620405], 
            ['Московская область', 55.8317203, 37.3295266], 
            ['Санкт-Петербург', 59.9391313, 30.3159004],
            ['Нижегородская область', 56.3240627, 44.0053913],
           ['Свердловская область', 56.8385216, 60.6054911],
            ['Ростовская область', 47.2224566, 39.718803],
              ['Красноярский край', 56.0093879, 92.8524806],
            ['Воронежская область', 51.6593332, 39.1969229],
              ['Иркутская область', 52.2864036, 104.2807466],
            ['Ставропольский край', 45.044516, 41.9689655],
           ['Ульяновская область', 54.3079415, 48.3748487],
            ['Мурманская область', 69.007721, 33.0685865],
         ['Волгоградская область', 48.7070042, 44.5170339],
           ['Саратовская область', 51.5303047, 45.9529348],
                ['Алтайский край', 51.9581028, 85.9603235],
              ['Хабаровский край', 48.4647258, 135.0598942],
          ['Оренбургская область', 51.7875092, 55.1018828],
         ['Новосибирская область', 55.028191, 82.9211489],
                ['Омская область', 54.9848566, 73.3674517],
            ['Краснодарский край', 45.0401604, 38.9759647],
             ['Самарская область', 53.1950306, 50.1069518],
               ['Приморский край', 43.1164904, 131.8823937],
           ['Кемеровская область', 55.3910651, 86.0467781],
               ['Республика Коми', 61.6686617, 50.8358151],
            ['Пензенская область', 53.1753314, 45.0348625],
                 ['Пермский край', 58.0102583, 56.2342034],
      ['Республика Саха (Якутия)', 62.0281405, 129.7325887],
           ['Республика Дагестан', 42.9849159, 47.5047181],
            ['Забайкальский край', 52.0340142, 113.4994],
               ['Томская область', 56.4845804, 84.9481582],
             ['Кировская область', 58.6035264, 49.6679304],
              ['Брянская область', 53.2419535, 34.3652146],
         ['Ленинградская область', 60.0191278, 30.6456718],
              ['Тульская область', 54.1920559, 37.6153842],
           ['Ярославская область', 57.6215477, 39.8977411],
             ['Орловская область', 52.9671298, 36.0696427],
          ['Белгородская область', 50.5976472, 36.5856652],
             ['Калужская область', 54.5059848, 36.2516245],
            ['Ивановская область', 56.9993792, 40.9728272],
               ['Курская область', 51.7303637, 36.1925603],
               ['Республика Крым', 44.9482948, 34.1001151],
              ['Тверская область', 56.8586059, 35.9116761],
             ['Рязанская область', 54.6254445, 39.7358609],
          ['Астраханская область', 46.3655652, 48.0559236],
            ['Республика Карелия', 61.7891264, 34.3596434],
            ['Тамбовская область', 52.7213154, 41.452264],
          ['Владимирская область', 56.1280804, 40.4084376],
         ['Удмуртская Республика', 56.852738, 53.2114896],
               ['Республика Тыва', 51.7191047, 94.4376882],
            ['Смоленская область', 54.7866913, 31.8152477],
             ['Псковская область', 57.8194415, 28.3317198],
           ['Республика Калмыкия', 46.3083344, 44.2702088],
            ['Республика Хакасия', 53.7223325, 91.4436721],
          ['Новгородская область', 58.5213846, 31.2755394],
              ['Республика Алтай', 51.9581028, 85.9603235],
           ['Костромская область', 40.9908264, 268617],
           ['Сахалинская область', 46.9591631, 142.737976],
           ['Вологодская область', 48.7070042, 44.5170339],
              ['Липецкая область', 52.610249, 39.5947883],
              ['Амурская область', 50.290659, 127.527198],
       ['Калининградская область', 54.7074702, 20.5073241],
           ['Республика Мордовия', 54.1809332, 45.1862632],
          ['Республика Татарстан', 49.1114975, 1216965],
          ['Республика Ингушетия', 43.1688611, 44.8131207],
            ['Курганская область', 55.4443883, 65.3161963],
             ['Республика Адыгея', 44.6098268, 40.1006606],
           ['Республика Марий Эл', 56.6343662, 47.8999706],
               ['Камчатский край', 53.036908, 158.6559254],
           ['Магаданская область', 59.5681332, 150.8084956],
                   ['Севастополь', 44.6167013, 33.525355],
          ['Чеченская Республика', 43.3180145, 45.698291],
     ['Ненецкий автономный округ', 67.6379672, 53.0069565],
    ['Чукотский автономный округ', 64.7313924, 177.5015421]]





In [9]:
#Создаём DataFrame с геоданными
city_loc_df = pd.DataFrame(city_loc)
city_loc_df = city_loc_df.rename(columns={city_loc_df.columns[0]: "city", 
                                          city_loc_df.columns[1]: "geo_lat", 
                                          city_loc_df.columns[2]: "geo_lon",})

In [10]:
#Объединяем DataFrame
total_tab = new_tr.merge(city_loc_df, how="inner", left_index=True, right_index=True)

total_tab.head()

# total - вероятность заболеть

,city_x,identified,recovered,died,new,recovered_day,died_day,population_density,population,area,federal district,total,city_y,geo_lat,geo_lon
0,Москва,1530989,1395285,26309,1639,3072,61,"4956,06",12692466,2561,ЦФО,0.012913,Москва,55.754047,37.620405
1,Санкт-Петербург,547704,519868,19757,1811,1972,49,"3843,90",5392992,1403,СЗФО,0.033581,Московская область,55.831720,37.329527
2,Московскаяобласть,400007,330540,7491,1309,1922,29,17342,7687647,44329,ЦФО,0.017027,Санкт-Петербург,59.939131,30.315900
3,Нижегородскаяобласть,145993,132727,5325,537,425,25,4181,3203818,76624,ПФО,0.016761,Нижегородская область,56.324063,44.005391
4,Ростовскаяобласть,114218,100373,5412,479,374,26,4155,4195327,100967,ЮФО,0.011417,Свердловская область,56.838522,60.605491


In [37]:
cities = total_tab
fig = px.scatter_mapbox(cities, lat="geo_lat", lon="geo_lon", hover_name="total", hover_data=["city_y", "new"],
                        color_discrete_sequence=["fuchsia"],size="new", zoom=3, height=500)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [38]:
#Сортируем DataFrame по вероятности заболеть и оставляем 1 наименьшее значение
safe = total_tab.sort_values(by='total')
safe.head(1)

,city_x,identified,recovered,died,new,recovered_day,died_day,population_density,population,area,federal district,total,city_y,geo_lat,geo_lon
61,РеспубликаТатарстан,24436,21594,637,49,39,4,5752,3902642,67847,ПФО,0.001256,Республика Мордовия,54.180933,45.186263
